In [ ]:
import pandas as pd
import numpy as np
import altair as alt

In [ ]:
big_foot = pd.read_csv("https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_data/main/bfro_reports_fall2022.csv")

In [ ]:
big_foot.columns

In [ ]:
big_foot.head()

In [ ]:
season_order = ['Spring', 'Summer', 'Fall', 'Winter', 'Unknown']

dropdown = alt.binding_select(options = season_order, name = "Season Selection: ")

selection = alt.selection_point(fields = ['season'], bind = dropdown)

color = alt.condition(
    selection,
    alt.Color('classification:N', title='Classification'),
    alt.value('lightgray')
)

opacity = alt.condition(
    selection,
    alt.value(1.0),
    alt.value(0.5)
)

chart1 = (
    alt.Chart(big_foot)
    .mark_bar()
    .encode(
        x=alt.X('season:N', title='Season', sort = season_order),
        y=alt.Y('count()', title='Number of Sightings'),
        color=color,
        opacity = opacity,
        tooltip=['season:N', 'classification:N', 'count():Q']
    )
    .properties(
        title='Bigfoot Sightings by Season and Classification',
        width=400
    )
    .add_params(
        selection
    )
)
chart1

chart2 = (
    alt.Chart(big_foot)
    .mark_bar()
    .encode(
        x=alt.X('state:N', title='Season'),
        y=alt.Y('count()', title='Number of Sightings'),
        color=alt.Color('classification:N', title='Classification'),
        tooltip=['state:N', 'classification:N', 'count():Q']
    )
    .properties(
        title='Bigfoot Sightings by Season and Classification',
        width=400
    ).transform_filter(
        selection
    )
)

chart = (chart1.properties(width=300) | chart2.properties(width=300))

chart

## Visualization 1: Bigfoot Sightings by Season and Classification

The dataset required minimal cleaning as Altair automatically handles missing values by excluding rows with NA values. The visualizations that I created only filter the data to focus on seasonal patterns, classification types, humidity, and temperature.

This visualization shows the distribution of Bigfoot sightings across different seasons, colored by classification type. I chose a stacked bar chart encoding because it effectively displays how sightings are distributed across both seasonal patterns and encounter classifications. The x-axis represents the four seasons, while the y-axis shows the count of sightings. I used a color scheme based on the classification of sightings for each classification type (Class A, Class B, Class C) to make it easy to distinguish between different types of encounters. The interactive dropdown filter allows users to select specific seasons, revealing how seasonal patterns vary geographically. This interactivitiy can help identify regional variation such as which season causes which specific states to have increased big foot sightings.


In [ ]:
myJekyllDir = '/Users/stevenwu/Desktop/StevenWu1.github.io/assets/json/'
chart.save(myJekyllDir + 'big_foot_season_state.json')

In [ ]:
brush = alt.selection_interval(encodings=['x', 'y'])

chart1 = (
    alt.Chart(big_foot)
    .mark_circle(size=60, opacity=0.7)
    .encode(
        x=alt.X('temperature_mid:Q', title='Average Temperature (°F)'),
        y=alt.Y('humidity:Q', title='Humidity (%)'),
        color=alt.Color('state:N', legend=None),
        tooltip=['state', 'county', 'temperature_mid', 'humidity', 'season']
    )
    .add_params(brush)
    .properties(
        title='Interactive Weather Conditions During Bigfoot Sightings',
        width=600,
        height=400
    )
)

from vega_datasets import data
states = alt.topo_feature(data.us_10m.url, feature='states')

background = alt.Chart(states).mark_geoshape(
    fill='lightgray',
    stroke='white'
).project('albersUsa').properties(
    width=500,
    height=300
)

chart2 = (
    alt.Chart(big_foot)
    .mark_circle(size=60)
    .encode(
        longitude='longitude:Q',
        latitude='latitude:Q',
        color=alt.Color('season:N'),
        tooltip=['state', 'county', 'season']
    )
    .transform_filter(brush)
    .properties(
        title='Map of Selected Bigfoot Sightings',
        width=500,
        height=300
    )
)

chart2 = background + chart2

chart = (chart1.properties(width=300) | chart2.properties(width=300))
chart

## Visualization 2: Bigfoot Sightings by State with Environmental Factors

For the second visualization, I created a coordinated view between a scatter plot and a map to explore environmental correlations with Bigfoot sightings. The scatter plot shows the relationship between temperature (x-axis) and humidity (y-axis), with points colored by state. The brush selection interactivity allows users to select points in the scatter plot based on environmental conditions, which then highlights the corresponding geographic locations on the map. This coordinated interaction helps users explore whether certain environmental conditions (like specific temperature and humidity ranges) correlate with sightings in particular geographic regions.

In [ ]:
chart.save(myJekyllDir + 'big_foot_humitity_temp_map.json')